In [21]:
#Finetune a Model with MobileNet in Python

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load MobileNet model without the top layer
base_model = MobileNet(weights='imagenet', include_top=False)

# Freeze the base model
for layer in base_model.layers:
    layer.trainable = False



C:\Users\Mahdi\AppData\Local\Temp\ipykernel_2088\1850288912.py:10: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNet(weights='imagenet', include_top=False)


In [25]:
# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)  # Assuming 10 classes

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [ ]:
# Data preparation
train_datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    './../images/test_task/source',  # Path to training data
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    './../images/test_task/target/',  # Path to training data
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)



In [ ]:
# Train the model
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

# Unfreeze some layers of the base model for fine-tuning
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Recompile the model
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)


Freezing layers in the base model means you’re not allowing those layers to learn or update their weights during training. This has a few benefits. First, if the base model's weights were trained on a very large dataset and already have a strong understanding of general features (like edges in images or common word patterns in text), freezing them allows the model to leverage this pre-learned knowledge and focus more on fine-tuning the new layers to your specific task. This can make training faster and prevent overfitting, since fewer parameters need updating. Essentially, it’s like using a well-trained athlete as a base and teaching them a new sport, rather than training them from scratch.

